# Hands-On: Counterfactual Explanations in Python

Popular Python packages for computing counterfactual explanations:
- [DiCE](https://github.com/interpretml/DiCE) -- general algorithms (incl. diversity)
- [CARLA](https://github.com/carla-recourse/CARLA) -- many algorithms (incl. causality)
- [Alibi](https://github.com/SeldonIO/alibi) -- Counterfactual Guided by Prototypes (outdated tensorflow version!)
- [CEML](https://github.com/andreArtelt/ceml) -- model-specific algorithms (incl. plausibility)
- ...

We consider the (toy) problem of explaining breast cancer predictions made by a random forest classifier.
In this context, we demonstrate how to
1. Implement a nearest neighbor CF baseline
2. Use DiCE for computing diverse CFs

In [ ]:
%pip install dice-ml scikit-learn matplotlib

In [ ]:
%pip install "numpy<2"

In [ ]:
from typing import Callable
import dice_ml
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.ensemble import RandomForestClassifier

In [ ]:
# Helper function for plotting CFs
def plot_barplot(x, labels):
    fig, ax = plt.subplots()
    ax.barh(labels, x, align='center', height=0.5)
    plt.show()

### Problem Setup

Breast cancer prediction using a random forest classifier.

1. Load data set
2. Split into train and test set
3. Train random forest classifier
4. Evalute classifier

#### 1. Load data set

In [ ]:
# Get feature names of breast cancer data set
df_data = load_breast_cancer(as_frame=True)
feature_names, target_names = list(df_data.feature_names), list(df_data.target_names)

feature_names, target_names

In [ ]:
# Load breast cancer data set
X, y = load_breast_cancer(return_X_y=True)

X.shape, y.shape

#### 2. Split into train and test set

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.3, shuffle=True)  # 30% test data

X_train.shape, y_train.shape, X_test.shape, y_test.shape

#### 3. Train random forest classifier

In [ ]:
clf = RandomForestClassifier()
clf.fit(X_train, y_train)

In [ ]:
y_train_pred = clf.predict(X_train)

#### 4. Evalute classifier

In [ ]:
y_test_pred = clf.predict(X_test)
f1_score(y_test, y_test_pred)

#### Pick samples to be explained

We want to explain all malignant predictions!

In [ ]:
# Negative classified samples -- i.e. y = 0 => malignant
idx = y_test_pred == 0
X_test_neg = X_test[idx, :]

In [ ]:
X_test_neg.shape

## A simple baselines: Nearest Neighbor Counterfactuals

Assuming a set of (labeled) samples is available -- e.g. training/validation/test data set. The Nearest Neighbor (also called *Nearest Unlike Neighbor*) Counterfactual corresponds to the closest sample from this test labeled with the resquested prediction:

$$\vec{x}_{cf} = \underset{\vec{x}_i\in D \mid y_i = y_{cf}}{\argmin} d(\vec{x}_{cf}, \vec{x}_i)$$

This simple method often constitures a surprisingly good baseline for plausible counterfactual explanations.

In [ ]:
# Different distance functions for measuring closeness/proximity to the original sample
def get_dist_func(desc: str, epsilon=1e-3) -> Callable[[np.ndarray, np.ndarray], float]:
    if desc == "l0":
        return lambda x_orig, x_cf: np.sum(np.abs(x_orig - x_cf) < epsilon)
    elif desc == "l1":
        return lambda x_orig, x_cf: np.sum(np.abs(x_orig - x_cf))
    elif desc == "l2":
        return lambda x_orig, x_cf: np.sum(np.square(x_orig - x_cf))
    else:
        raise ValueError(f"Unknown distance function '{desc}'")

In [ ]:
# Nearest Neighbor Counterfactual
class NearestNeighborCF():
    def __init__(self, clf, X_train: np.ndarray, y_train: np.ndarray, dist="l1"):
        self.clf = clf

        y_pred = clf.predict(X_train)
        mask = y_pred == y_train    # Limit the feasible set to correctly classified samples, assuming the label is available
        self.X = X_train[mask, :]
        self.y = y_train[mask]

        self.dist = dist
        if not callable(self.dist):
            self.dist = get_dist_func(dist)

    def compute_counterfactual(self, x_orig: np.ndarray, y_target: int) -> np.ndarray:
        # Potential target samples
        mask = self.y == y_target
        X_ = self.X[mask, :]

        # Find closest samples with the requested prediction
        X_diff = X_ - x_orig
        dist = [self.dist(x_orig, X_[i, :]) for i in range(X_diff.shape[0])]
        idx = np.argmin(dist)

        return X_[idx, :]

    def compute_delta_cf(self, x: np.ndarray, y_target: int) -> np.ndarray:
        return self.compute_counterfactual(x, y_target) - x
    
    def compute_counterfactual_batch(self, X_orig: np.ndarray, y_target: np.ndarray) -> np.ndarray:
        X_cf = []

        for i in range(X_orig.shape[0]):
            X_cf.append(self.compute_counterfactual(X_orig[i, :], y_target[i]))

        return np.array(X_cf)
    
    def compute_delta_cf_batch(self, X_orig: np.ndarray, y_target: np.ndarray) -> np.ndarray:
        Delta_cf = []

        for i in range(X_orig.shape[0]):
            Delta_cf.append(self.compute_counterfactual(X_orig[i, :], y_target[i]) - X_orig[i, :])
        
        return np.array(Delta_cf)

#### Compute Counterfactual Explanations

In [ ]:
cf_baseline = NearestNeighborCF(clf, X_train, y_train)

In [ ]:
X_cf = cf_baseline.compute_counterfactual_batch(X_orig=X_test_neg,
                                                y_target=np.array([1] * X_test_neg.shape[0]))  # Flip prediction from 0 -> 1 for every query sample

Inspect first counterfactual

In [ ]:
X_cf[0, :]

Compute and inspect changes delta

In [ ]:
Delta_cf = cf_baseline.compute_delta_cf_batch(X_orig=X_test_neg,
                                              y_target=np.array([1] * X_test_neg.shape[0]))

In [ ]:
Delta_cf[0, :]

In [ ]:
np.round(Delta_cf[0, :], 1)

Visualize the counterfactual

In [ ]:
plot_barplot(Delta_cf[0, :], feature_names)

## DiCE: Diverse Counterfactual Explanations

#### Pre-requisits

1. Predictive model (e.g. classifier)
2. (Training) data set (as a pandas [DataFrame](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.html))
3. Query sample(s) -- i.e. sample(s) for which we want to generate a CF

#### How to use DiCE

1. Wrap (training) data set as a [dice_ml.Data](https://interpret.ml/DiCE/dice_ml.html#dice_ml.data.Data) instance
2. Wrap predictive model as a [dice_ml.Model](https://interpret.ml/DiCE/dice_ml.html#dice_ml.model.Model) instance
3. Wrap query samples as a pandas [DataFrame](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.html)
4. Compute CFs by using creating a [dice_ml.Dice](https://interpret.ml/DiCE/dice_ml.html#dice_ml.dice.Dice) instance and calling the [generate_counterfactuals()](https://interpret.ml/DiCE/dice_ml.explainer_interfaces.html#dice_ml.explainer_interfaces.explainer_base.ExplainerBase.generate_counterfactuals) function

#### 1. Wrap (training) data set as a [dice_ml.Data](https://interpret.ml/DiCE/dice_ml.html#dice_ml.data.Data) instance

In [ ]:
X_df = pd.DataFrame(X_train, columns=feature_names)
y_df = pd.DataFrame(y_train_pred, columns=["y"]).astype(np.int32)  # Alternative, use only correctly classified samples!
data_df = pd.concat([X_df, y_df], axis=1)

data = dice_ml.Data(dataframe=data_df,
                    continuous_features=feature_names,  # All feature are continous!
                    outcome_name='y')

#### 2. Wrap predictive model as a [dice_ml.Model](https://interpret.ml/DiCE/dice_ml.html#dice_ml.model.Model) instance

In [ ]:
model = dice_ml.Model(model=clf, backend='sklearn')

#### 3. Wrap query samples as a pandas [DataFrame](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.html)

In [ ]:
X_queries = pd.DataFrame(X_test_neg, columns=feature_names)

#### 4. Compute CFs by using creating a [dice_ml.Dice](https://interpret.ml/DiCE/dice_ml.html#dice_ml.dice.Dice) instance and calling the [generate_counterfactuals()](https://interpret.ml/DiCE/dice_ml.explainer_interfaces.html#dice_ml.explainer_interfaces.explainer_base.ExplainerBase.generate_counterfactuals) function

In [ ]:
cf_algo = dice_ml.Dice(data, model, method="genetic")   # Evolutionary algorithm for computing CFs -- see documentation for alternatives!

cf_results = cf_algo.generate_counterfactuals(X_queries,
                                              total_CFs=3,        # 3 diverse CFs
                                              desired_class="opposite",  # Flip prediction!
                                              verbose=False)

In [ ]:
cf_results

In [ ]:
#cf_results.visualize_as_dataframe()

In [ ]:
len(cf_results.cf_examples_list)

Inspect CFs of the first query sample

In [ ]:
cf_results.cf_examples_list[0].final_cfs_df

Export to [NumPy arrays](https://numpy.org/doc/stable/reference/generated/numpy.array.html)

In [ ]:
X_cf = cf_results.cf_examples_list[0].final_cfs_df[feature_names].to_numpy() 

In [ ]:
X_cf

Compute change vector

In [ ]:
X_cf - X_test_neg[0, :] 

In [ ]:
np.round(X_cf - X_test_neg[0, :], 1)

Visualize the counterfactual (i.e. change vector)

In [ ]:
plot_barplot(X_cf[0, :] - X_test_neg[0, :], feature_names)